<a href="https://colab.research.google.com/github/ReaganJHarris/ET_Retrieval/blob/main/ETa_Statistics_csvExport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Search **check_replace_this_var** in the notebook and correct the values  to create folder/imageCollection structures that represent your year and landsat. 

*   ARDretrieval_iDate_Col
*   cur_ImgCol
*   ETretrieval_Image_path
*   asset_folder

In [ ]:
import os
import sys
import datetime
from time import strftime, strptime, gmtime
import math
import pprint
import copy
import numpy as np

In [ ]:
!pip install 'pyOpenSSL>=0.11'
!pip install earthengine-api

     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 3.6 MB 48.2 MB/s 


In [ ]:
!earthengine authenticate

import ee
ee.Initialize()

image = ee.Image('srtm90_v4')
print(image.getInfo())

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=fLxN0FwouRNoWIfO4Taw63uMakr7YbtYDh7An9Hpzco&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhJD4UgRiHOu6umoQ2IOpMTkpisJ77LfU7c55hpr-dSN-e8tS2dWb4

Successfully saved authorization token.
{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
from google.colab import auth
auth.authenticate_user()

# Image/Feature Collections

In [ ]:
Counties = ee.FeatureCollection("TIGER/2018/Counties")

ARD_ET_8 = ee.ImageCollection("projects/ColoradoView/Landsat/ARD_ET/8")
ARD_ET_7 = ee.ImageCollection("projects/ColoradoView/Landsat/ARD_ET/7")
ARD_ET_5 = ee.ImageCollection("projects/ColoradoView/Landsat/ARD_ET/5")

# Constants

## Masks

In [ ]:
#@title ARD mask
# Validation  
#TODO: check the function, so it applies to L8, L7, and L5
def maskLandsatARD_ET(image): 
  fillBitMask = 1
  clearBitMask = 1 << 1
  cloudBitMask = 1 << 5
  # var cloudConfBit6Mask = 1 << 6;
  # var cloudConfBit7Mask = 1 << 7;
  # var cirrusConfBit8Mask = 1 << 8;
  # var cirrusConfBit9Mask = 1 << 9;
  
  ETa_name = 'b1' #Et ... TODO
  ETf_name = 'b2'
  ETun_name = 'b3'
  pixelqa_name = 'b4'
  
  ETqa_threshold = 1.0 #1-2 mm
  
  #  get the pixel QA band
  #  pixelqa_name: 'b2'
  pixelqa = image.select(pixelqa_name) #
  
  #  get the STQA band
  # lstqa = image.select(lstqa_name)
  # stqa_scale = ee.Number(image.get(lstqa_scale_name))
  
  mask = pixelqa.bitwiseAnd(fillBitMask).eq(0)  \
    .And(pixelqa.bitwiseAnd(clearBitMask).neq(0)) \
    .And(pixelqa.bitwiseAnd(cloudBitMask).eq(0))
        #.and(pixelqa.bitwiseAnd(cloudConfBit7Mask).eq(0))
        #.and(pixelqa.bitwiseAnd(cirrusConfBit9Mask).eq(0))
        # .and(lstqa.multiply(stqa_scale).lt(lstqa_threshold))
        
        
  #  Return the masked image (excluding the PIXELQA and STQA layers)
  return image.updateMask(mask).select(ETa_name)
      #  .updateMask(mask_1);// [lst_name, pixelqa_name, lstqa_name]

## Functions

In [ ]:
def check_asset_existence(asset_id_str: str) -> bool:
  '''determine whether the given asset_id_str is a valid ee asset.'''
  try:
    ret_info = ee.data.getAsset(asset_id_str)
    return True
  except ee.ee_exception.EEException as err:
    return False

In [ ]:
def generate_ET_retrieval_asset_id(
  root_path: str, county_name: str, date: str) -> str:
  '''generate the asset id (string) from the given path, county, and date '''
  return os.path.join(root_path, county_name, date)

# Need to enter in date str when calling this function

### Statistics functions

In [ ]:

# function statistics_get_count(in_Image, Bnds, scale=30) {
def statistics_get_count(in_Image, Bnds, scale):
  in_Image = ee.Image(in_Image)
  area = in_Image.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=Bnds.first().geometry(),
    scale=scale, #30, #scale,
    maxPixels=1e13
  ).values().get(0)
  return area


def statistics_get_mean(in_Image, Bnds, scale):
  in_Image = ee.Image(in_Image)
  mean = in_Image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=Bnds.first().geometry(),
    scale=scale,
    maxPixels=1e13
  ).values().get(0)
  return mean


def statistics_get_median(in_Image, Bnds, scale):
  in_Image = ee.Image(in_Image)
  median = in_Image.reduceRegion(
    reducer=ee.Reducer.median(),
    geometry=Bnds.first().geometry(),
    scale=scale,
    maxPixels=1e13
  ).values().get(0)
  return median


def statistics_get_Std_Dev(in_Image, Bnds, scale):
  in_Image = ee.Image(in_Image)
  Std_Dev = in_Image.reduceRegion(
    reducer=ee.Reducer.stdDev(),
    geometry=Bnds.first().geometry(),
    scale=scale,
    maxPixels=1e13
  ).values().get(0)
  return Std_Dev


def statistics_get_min(in_Image, Bnds, scale):
  in_Image = ee.Image(in_Image)
  min = in_Image.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=Bnds.first().geometry(),
    scale=scale,
    maxPixels=1e13
  ).values().get(0)
  return min


def statistics_get_max(in_Image, Bnds, scale):
  in_Image = ee.Image(in_Image)
  max = in_Image.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=Bnds.first().geometry(),
    scale=scale,
    maxPixels=1e13
  ).values().get(0)
  return max

def statistics_get_R_square(in_Image_1, in_Image_2, Bnds, scale):
  Img1 = ee.Image(in_Image_1)
  Img2 = ee.Image(in_Image_2)
  ImgStk = Img1.select(0).rename('b1').addBands(Img2.select(0).rename('b2'))

  pearsonsCorrelation = ImgStk.reduceRegion(
    reducer=ee.Reducer.pearsonsCorrelation(),
    geometry=Bnds.first().geometry(),
    scale=scale,
    maxPixels=1e13
    )
  Correlation = pearsonsCorrelation.get('correlation')
  R = ee.Number(Correlation)
  R_squared = R.pow(2) #R squared
  return R_squared


#  RMSE
def statistics_get_RMSE(in_Image_1, in_Image_2, Bnds, scale):
  Img_diff = in_Image_1.subtract(in_Image_2).rename('Img_diff')
  
  diff_squared = Img_diff.expression('Img_diff**2',{
  'Img_diff': Img_diff
  })

  RMSE = ee.Number(diff_squared.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=Bnds.first().geometry(),
    scale=scale,
    maxPixels=1e13
  ).values().get(0)).sqrt()
  return RMSE


In [ ]:
#@title Create an empty list to hold dictionaries for each date


statistics_key_list = ee.List([
  'Date_string',
  'area_ARD','area_retrieved_ET',
  'mean_ARD','mean_retrieved_ET',
  'median_ARD','median_retrieved_ET',
  'Std_Dev_ARD','Std_Dev_retrieved_ET',
  'min_ARD','min_retrieved_ET',
  'max_ARD','max_retrieved_ET',
  'R_square',
  'RMSE'])

def metric_dictionary_not_null(date, ETretrieval_iDate, ARDretrieval_iDate_Col, TgtCounty_Bnds, scale):

  # print('date:', date)
  ETretrieval_iDate = ee.Image(ETretrieval_iDate)
  ARDretrieval_iDate = ee.Image(ARDretrieval_iDate_Col)
  # var LSTretrieval_iDate =  denverLST_retrieval.filterDate(ee.Date(date)).first();
  # var ARDretrieval_iDate = ARD_LST.map(maskARDL8lst).filterDate(ee.Date(date))
    # .mosaic()//composites images in collection using mask
    # .clipToCollection(TgtCounty_Bndry)//clips to this set boundary
    # .multiply(0.1);//scaling factor for ARD_LST

  # Masked images with updated variable names
  ARD_Img_mask = ARDretrieval_iDate.mask()
  ETretrieval_Mask = ETretrieval_iDate.mask().clip(TgtCounty_Bnds)
  
  # print("ETretrieval_Mask", ETretrieval_Mask)

  # Combined mask - retrieved LST and ARD with new variables defined in function
  combined_R_A_mask = ETretrieval_Mask.And(ARD_Img_mask)
  ET_for_validation = ETretrieval_iDate.updateMask(combined_R_A_mask) #updates mask for LST_retrieval
  ARD_Img_for_validation = ARDretrieval_iDate.updateMask(combined_R_A_mask) #updates mask for ARD8_Img
    
  # print('Mask_updates:', ARDretrieval_iDate, ETretrieval_Mask)
  
 
  # Statistics Calculations begin
  area_ARD = statistics_get_count(ARD_Img_for_validation, TgtCounty_Bnds, scale)
  area_retrieved_ET = statistics_get_count(ET_for_validation, TgtCounty_Bnds, scale) # TgtCounty_Bndry, scale=scale);
  
  # print('area_ARD', area_ARD)
  # print('area_retrieved_ET', area_retrieved_ET)
  
  mean_ARD = statistics_get_mean(ARD_Img_for_validation, TgtCounty_Bnds, scale)
  mean_retrieved_ET = statistics_get_mean(ET_for_validation, TgtCounty_Bnds, scale)
  # print('mean_ARD', mean_ARD);
  # print('mean_retrieved_LST', mean_retrieved_LST);
  
  median_ARD = statistics_get_median(ARD_Img_for_validation, TgtCounty_Bnds, scale)
  median_retrieved_ET = statistics_get_median(ET_for_validation, TgtCounty_Bnds, scale)
  # print('median_ARD', median_ARD);
  # print('median_retrieved_LST', median_retrieved_LST);
  
  Std_Dev_ARD = statistics_get_Std_Dev(ARD_Img_for_validation, TgtCounty_Bnds, scale)
  Std_Dev_retrieved_ET = statistics_get_Std_Dev(ET_for_validation, TgtCounty_Bnds, scale)
  # print('Std_Dev_ARD',Std_Dev_ARD);
  # print('Std_Dev_retrieved_LST', Std_Dev_retrieved_LST);
  
  min_ARD = statistics_get_min(ARD_Img_for_validation, TgtCounty_Bnds, scale)
  min_retrieved_ET = statistics_get_min(ET_for_validation, TgtCounty_Bnds, scale)
  # print('min_ARD',min_ARD);
  # print('min_retrieved_LST', min_retrieved_LST);
  
  max_ARD = statistics_get_max(ARD_Img_for_validation, TgtCounty_Bnds, scale)
  max_retrieved_ET = statistics_get_max(ET_for_validation, TgtCounty_Bnds, scale)
  # print('max_ARD',max_ARD);
  # print('max_retrieved_LST', max_retrieved_LST);
  
  R_square = ee.Algorithms.If(
    ee.Number(area_retrieved_ET).eq(0), 
    # null,
    None,
    statistics_get_R_square(ET_for_validation, ARD_Img_for_validation, TgtCounty_Bnds, scale)) #R^2 between retrieved LST and ARD8
  # print('R_square',R_square)
  
  RMSE = ee.Algorithms.If(
    ee.Number(area_retrieved_ET).eq(0),
    # null,
    None,
    statistics_get_RMSE(ET_for_validation, ARD_Img_for_validation, TgtCounty_Bnds, scale))
  # print('RMSE', RMSE)

  # Assign the values that will be returned to matching previous stringList values 
  # in that format
  # statistics_value_list = ee.List([
  #   date, 
  #   area_ARD8,area_retrieved_LST,
  #   mean_ARD8,mean_retrieved_LST,
  #   median_ARD8,median_retrieved_LST,
  #   Std_Dev_ARD8,Std_Dev_retrieved_LST,
  #   min_ARD8,min_retrieved_LST,
  #   max_ARD8,max_retrieved_LST,
  #   R_square,
  #   RMSE])
  
  statistics_value_list = ee.List([
    date, 
    area_ARD,area_retrieved_ET,
    mean_ARD,mean_retrieved_ET,
    median_ARD,median_retrieved_ET,
    Std_Dev_ARD,Std_Dev_retrieved_ET,
    min_ARD,min_retrieved_ET,
    max_ARD,max_retrieved_ET,
    R_square,
    RMSE])
  
  # Create a dictionary from two previously defined lists
  statistics_dict = ee.Dictionary.fromLists(statistics_key_list,statistics_value_list)
  # print('Stats_dict', statistics_dict );
  
  # Map.addLayer(LSTretrieval_iDate, {}, "LSTretrieval_iDate");
  
  # Map.addLayer(ARD8retrieval_iDate, {}, "ARD8retrieval_iDate");
  
  # Map.addLayer(LST_for_validation, {}, "LST_for_validation");
  
  # Map.addLayer(ARD8_Img_for_validation, {}, "ARD8_Img_for_validation");
  
  return statistics_dict




In [ ]:
#@title Function retrieving values for each statType
def metric_dictionary(date, TgtCounty_Bnds, scale, ETretrieval_Image_path):
  
  # Create format of key that list will be returned in

  # empty_val_list = ee.List([
  #   date, 
  #   # null, null, null, null, null, null, null, null, null, null, null, null, null, null
  #   None, None, None, None, None, None, None, None, None, None, None, None, None, None
  #   ])

  # empty_statistics_dict = ee.Dictionary.fromLists(statistics_key_list, empty_val_list)

  # print('date:', date)
  
  # # LSTretrieval_IC_iDate = sanJuanLST_retrieval.filterDate(ee.Date(date))
  # ETreteival_Image_path = os.path.join(ET_Retrieval_rootPath, countyName, date)
  # if ETreteival_Image_path not exists:
  #   # LSTretrieval_IC_iDate = ee.ImageCollection(ee.List([]))
  #   flag_retrieval = ee.Number(0)
    
  # else:
  #   flag_retrieval = ee.Number(1)
  #   LSTretrieval_iDate = ee.Image(ETreteival_Image_path) #obtain image from line 399

  # LSTretrieval_iDate = ee.Algorithms.If(
  #   ee.Number(LSTretrieval_IC_iDate.size()).eq(0),
  #   ee.Image(0).updateMask(0),
  #   ee.Image(ETreteival_Image_path)
  #   )

  ETretrieval_iDate = ee.Image(ETretrieval_Image_path) #TODO...change back to ETretrieval_Image_path

  
  # var LSTretrieval_iDate =  denverLST_retrieval.filterDate(ee.Date(date)).first();
  # LSTretrieval_iDate = LSTretrieval_IC_iDate.first()
  # var ARDretrieval_iDate = ARD_LST.map(maskARDL8lst).filterDate(ee.Date(date))
    # .mosaic()//composites images in collection using mask
    # .clipToCollection(TgtCounty_Bndry)//clips to this set boundary
    # .multiply(0.1);//scaling factor for ARD_LST
  

  # print("Lst_ret_date:", LSTretrieval_iDate);
  # print("ee.Number(LSTretrieval_IC_iDate.size()).gte(1)", ee.Number(LSTretrieval_IC_iDate.size()).gte(1));
  
  # 
  # LSTretrieval_iDate = ee.Algorithms.If(
  #   ee.Number(LSTretrieval_IC_iDate.size()).eq(0),
  #   ee.Image(0).updateMask(0),
  #   LSTretrieval_iDate
  #   )
  
  # print("updated ET_ret_date:", ETretrieval_iDate)

  ## TODO :Test mask
  #TODO....change back to argument (date)
  #composites images in collection using mask 
  #clips to this set boundary
  ARDretrieval_iDate_Col = ARD_ET_5.map(maskLandsatARD_ET).filterDate(ee.Date(date))  \
    .mosaic() \
    .clipToCollection(TgtCounty_Bnds)  \
    .multiply(0.001) #scaling factor for ARD_ET
  
    
  return metric_dictionary_not_null(
    date, ETretrieval_iDate, ARDretrieval_iDate_Col, TgtCounty_Bnds, scale)




# Single Day


In [ ]:
#@title County Geometry
# parameters
TgtCounty_index = 0;
countyGEOID_array = ['08031','08111','08047','08081','08039','08095'];
countyName_array = ['Denver','San_Juan', 'Gilpin','Moffat','Elbert','Phillips'];


TgtCounty_GEOID = countyGEOID_array[TgtCounty_index]
TgtCounty_Name = countyName_array[TgtCounty_index]

# get the State boundary of Colorado
# State_Bnds_CO = State_Bnds.filter(ee.Filter.eq('GEOID', '08')) #CO:'08'; UT:'49'
# print(State_Bnds_CO)

# target county
TgtCounty_Bnds = Counties.filter(ee.Filter.eq('GEOID', TgtCounty_GEOID))
# print('TgtCounty_Bndry', TgtCounty_Bnds)

# get the centroid of the county
TgtCounty_Bnds_geom = TgtCounty_Bnds.first().geometry()
TgtCounty_centroid = TgtCounty_Bnds_geom.centroid()
# print('TgtCounty_centroid', TgtCounty_centroid)

total_count_TgtCounty_500 = ee.Image(1).clipToCollection(TgtCounty_Bnds).rename('count').reduceRegion(
      reducer=ee.Reducer.count(),
      geometry=TgtCounty_Bnds.first().geometry(),
      scale=500,
      maxPixels=1e13 
    ).get('count')


In [ ]:
list_metrics = [] #ee.List([])
scale=30 #scale variable get

ET_Retrieval_rootPath = 'projects/ColoradoView/ET_Retrieval'

In [ ]:
#@title Valid Dates
# dates
start_date_str = '2018-01-01' #'2014-01-01'
end_date_str = '2018-12-31' # '2014-01-31'

# dates  
start_date = ee.Date(start_date_str)
end_date = ee.Date(end_date_str)
date_range = end_date.difference(start_date, 'day').getInfo()

#  create list of dates from start through date range
days = ee.List.sequence(0, date_range, 1.0)
def generate_date_feature(d):
# date_Fts = days.map(function(d) {
  #  d: the index of the continuous sequence (i.e., days)
  #  cur_date: ee.Date of the day pointed by d
  cur_date = start_date.advance(d, 'day')
  #  filter by the cur_date
  cur_ImgCol = ARD_ET_8.filterDate(cur_date)
  #  reduce resolution of the ImageCollection 
  # cur_ImgCol = cur_ImgCol.map(function(inImg){
  def coarse_reproject(inImg):
    # reproject in very low resolution (1920m) in order to reduce the calulation burdun.
    return inImg.reproject('EPSG:3857', None, 500)
  
  cur_ImgCol = cur_ImgCol.map(coarse_reproject)
  # mosaic ImageCollection and clip to the target boundary
  cur_ImgMosaic = ee.Image(cur_ImgCol.mosaic()).clipToCollection(TgtCounty_Bnds)
  # get the number of bands in the Image
  num_bands = cur_ImgMosaic.bandNames().size()
  # If number of bands > 0, return a normal date string, otherwise get an empty sting
  cur_date_str = ee.Algorithms.If(num_bands.gt(0), cur_date.format('YYYY-MM-dd'), '')

  cur_countMaskVal = ee.Algorithms.If(num_bands.gt(0), 
    cur_ImgMosaic.mask().clipToCollection(TgtCounty_Bnds).select(0).updateMask(cur_ImgMosaic.mask().select(0)).rename('count').reduceRegion( 
      reducer= ee.Reducer.count(), 
      geometry= TgtCounty_Bnds.first().geometry(), 
      scale= 500, 
      maxPixels= 1e13 
    ).get('count'), 
    0)
  
  # create a empty Feature with two properties: date_str and date_str_len
  ret_date_Ft = ee.Feature(None, { 
  'date_str': cur_date_str, 
  'date_str_len': ee.String(cur_date_str).length(), 
  'count': cur_countMaskVal})
  return ret_date_Ft

date_Fts = days.map(generate_date_feature)
# print("date_Fts", date_Fts);
 
 
valid_dates_filtered = date_Fts \
  .filter(ee.Filter.gt('date_str_len', 0)) \
  .filter(ee.Filter.gt('count', ee.Number(total_count_TgtCounty_500).multiply(0.5)))
# print("valid_dates_Fts", valid_dates_filtered)

def features_to_string(inFt):
  inFt = ee.Feature(inFt)
  return inFt.get('date_str')

valid_dates_filtered_StringList = valid_dates_filtered.map(features_to_string)

valid_dates = valid_dates_filtered_StringList.getInfo()
# num_valid_dates = valid_dates.length
num_valid_dates = len(valid_dates)
print(valid_dates)


['2018-01-02', '2018-01-11', '2018-03-16', '2018-04-01', '2018-04-08', '2018-04-17', '2018-05-26', '2018-06-04', '2018-06-11', '2018-06-20', '2018-06-27', '2018-07-06', '2018-07-13', '2018-08-07', '2018-08-14', '2018-08-23', '2018-08-30', '2018-09-08', '2018-09-15', '2018-10-26', '2018-11-02', '2018-11-18', '2018-12-04', '2018-12-13', '2018-12-20', '2018-12-29']


In [ ]:
#@title Checks existence of valid dates and calls stat functions in to ee.List
# Converts to string list of valid dates & creates
# variable for numValiddates = validdates.length
valid_dates = valid_dates_filtered_StringList.getInfo()
# num_valid_dates = valid_dates.length
num_valid_dates = len(valid_dates)
print('valid_dates', valid_dates)

# Create empty list that will hold dictionaries for each date
list_metrics = [] #ee.List([])

# forloop iterating over validates and appending every stat 
# value to empty dictionary(list) for EACH valid date
# for (iValidDates = 0; iValidDates < num_valid_dates; iValidDates++):
for iValidDates in list(range(num_valid_dates)):

  date = valid_dates[iValidDates]
  # print(date)
  ## new ##

  ## TODO: prepare county_name from county meta dict {geom_id: county_name, ...}
  # county_name = {'08031':'Denver',
                #  '08039':'Elbert',
                #  '08047':'Gilpin',
                #  '08081':'Moffat',
                #  '08095':'Phillips',
                #  '08111':'San_juan'
                #  }

  #TODO: adjust asset id to have landsat sensor in the parth
  ETretrieval_Image_path =  \
    generate_ET_retrieval_asset_id(ET_Retrieval_rootPath, countyName_array[TgtCounty_index], date)

  # break

  flag_exist = check_asset_existence(str(ETretrieval_Image_path))
  print(date, flag_exist)

  if not flag_exist:
    print(
      f"Warning: ET retrieval for {countyName_array[TgtCounty_index]} on {date} doesn't exist. "
      "Skip the date."
    )
    continue

  # if flag_exist == True:
  #   valid_ETretrieval_Image_path = flag_exist
    # print(valid_ETretrieval_Image_path)
    #########
  metric = metric_dictionary( \
    date, TgtCounty_Bnds, scale, ETretrieval_Image_path)   

  list_metrics.append(metric)

# raise ValueError()

# print("Metric list: ", list_metrics)

# Variable converting empty 'list_metrics' into an ee.List
ee_list_metrics = ee.List(list_metrics)
# print("ee_list_metrics: ", ee_list_metrics)




In [ ]:
#@title Creates feature collection
#  
# New variable mapping an algorithm(function) over ee.List (metrics) 
# w/ input & output
def create_feature(metricDictionary):
  # Creates a feature from ee_list_metrics
  ret_ft = ee.Feature(None, metricDictionary)
  return ret_ft

ee_listMetrics_fts = ee_list_metrics.map(create_feature)

# ee_listMetrics_fts = ee_list_metrics.map(function(metricDictionary){
  # Creates a feature from ee_list_metrics
#   var ret_ft = ee.Feature(null, metricDictionary);
#   return ret_ft;
# });



# print('metric_to_ftCollection', metric_to_ftCollection)

# Converts list of dictionaries into a feature collection FOR EXPORT
metric_to_ftCollection = ee.FeatureCollection(ee_listMetrics_fts)
# print('metric_to_ftCollection', metric_to_ftCollection)



In [ ]:
#@title Exports Metrics table as .csv
desc1 = 'Metrics_table_' + countyName_array[TgtCounty_index] + '_' + start_date_str + '-' + end_date_str
# TODO ...
# include GEOID, County name
task = ee.batch.Export.table.toDrive(
  collection=metric_to_ftCollection, 
  description=desc1, 
  fileNamePrefix=desc1,
  folder='ET_retrieval_Metrics',
  fileFormat='CSV'
)

task.start()


# Multi Day


In [ ]:
#@title Function for valid dates and county geometry
# for key, values in GEOID.items():
#def FIPS_valid_dates(county):
def parameter_fn(county_dict, TgtCounty_GEOID, start_date_str):

  #start_date_str = '2017-01-01'
  #end_date_str = '2017-12-31'

  Date_st = ee.Date(start_date_str)
  #Date_ed = ee.Date(end_date_str)
  Date_ed = Date_st.advance(1, 'year')

  date_range = Date_ed.difference(Date_st, 'day').getInfo()

  # get the State boundary of Colorado
  # TgtCounty_index = 0

  #TgtCounty_GEOID = countyGEOID_array #[TgtCounty_index] #08123：	Weld 
  # look up county name in the dict using the key (GEOID)
  if (TgtCounty_GEOID in county_dict):
    #TgtCounty_Name = countyName_array #[TgtCounty_index]
    TgtCounty_Name = county_dict[TgtCounty_GEOID]
  else:
    # GEOID not found
    raise ValueError(f"Error: given GEOID {TgtCounty_GEOID} not found.")


  TgtCounty_Bnds = Counties.filter(ee.Filter.eq('GEOID', TgtCounty_GEOID))
  # print(TgtCounty_Bnds)
  # print(TgtCounty_Bnds.first().geometry().getInfo())
  TgtCounty_Bnds_geom = TgtCounty_Bnds.first().geometry()
  TgtCounty_centroid = TgtCounty_Bnds_geom.centroid()
  
  geom_State_Bnds_CO_geometries = TgtCounty_Bnds.first().geometry().getInfo()
  # print(geom_State_Bnds_CO_geometries)
  # if isinstance(geom_State_Bnds_CO_geometries, list):
  if geom_State_Bnds_CO_geometries['type'] == 'GeometryCollection':
    for igeom in geom_State_Bnds_CO_geometries['geometries']:
      if igeom['type'] == 'Polygon' or igeom['type'] == 'MultiPolygon':
        geom_State_Bnds_CO = igeom['coordinates']
        break
  else:
    geom_State_Bnds_CO = geom_State_Bnds_CO_geometries['coordinates']

  # print(geom_State_Bnds_CO)
  # geom_State_Bnds_CO = TgtCounty_Bnds.first().geometry().getInfo()['coordinates'] 

  geom_str = str(geom_State_Bnds_CO)
                
  total_count_TgtCounty_500 = ee.Image(1).clipToCollection(TgtCounty_Bnds).rename('count').reduceRegion( \
        reducer= ee.Reducer.count(), \
        geometry= TgtCounty_Bnds.first().geometry(), \
        scale= 500, \
        maxPixels= 1e13 \
      ).get('count')

  ## valid dates function
  date_range = Date_ed.difference(Date_st, 'day').getInfo();

  # create list of dates from start through date range
  days = ee.List.sequence(0, date_range, 1.0)
  def generate_date_feature(d):
  # date_Fts = days.map(function(d) {
    #  d: the index of the continuous sequence (i.e., days)
    #  cur_date: ee.Date of the day pointed by d
    cur_date = Date_st.advance(d, 'day')
    #  filter by the cur_date
    cur_ImgCol = ARD_ET_5.filterDate(cur_date)
    #  reduce resolution of the ImageCollection 
    # cur_ImgCol = cur_ImgCol.map(function(inImg){
    def coarse_reproject(inImg):
      # reproject in very low resolution (1920m) in order to reduce the calulation burdun.
      return inImg.reproject('EPSG:3857', None, 500)
    
    cur_ImgCol = cur_ImgCol.map(coarse_reproject)
    # mosaic ImageCollection and clip to the target boundary
    cur_ImgMosaic = ee.Image(cur_ImgCol.mosaic()).clipToCollection(TgtCounty_Bnds)
    # get the number of bands in the Image
    num_bands = cur_ImgMosaic.bandNames().size()
    # If number of bands > 0, return a normal date string, otherwise get an empty sting
    cur_date_str = ee.Algorithms.If(num_bands.gt(0), cur_date.format('YYYY-MM-dd'), '')

    cur_countMaskVal = ee.Algorithms.If(num_bands.gt(0), 
      cur_ImgMosaic.mask().clipToCollection(TgtCounty_Bnds).select(0).updateMask(cur_ImgMosaic.mask().select(0)).rename('count').reduceRegion( 
        reducer= ee.Reducer.count(), 
        geometry= TgtCounty_Bnds.first().geometry(), 
        scale= 500, 
        maxPixels= 1e13 
      ).get('count'), 
      0)
    
    # create a empty Feature with two properties: date_str and date_str_len
    ret_date_Ft = ee.Feature(None, { 
    'date_str': cur_date_str, 
    'date_str_len': ee.String(cur_date_str).length(), 
    'count': cur_countMaskVal})
    return ret_date_Ft

  date_Fts = days.map(generate_date_feature)
  # print("date_Fts", date_Fts)

  # filter List of Features to exclude those invalid dates (i.e., date_str_len == 0)
  valid_dates_filtered = date_Fts \
    .filter(ee.Filter.gt('date_str_len', 0)) \
    .filter(ee.Filter.gt('count', ee.Number(total_count_TgtCounty_500).multiply(0.5)))
  # print("valid_dates_Fts", valid_dates_filtered)

  # convert the List of Features to List of Strings (i.e., list of date_str_len)
  def features_to_string(inFt):
    inFt = ee.Feature(inFt)
    return inFt.get('date_str')

  valid_dates_filtered_StringList = valid_dates_filtered.map(features_to_string)
  # batch fetch List content to client
  valid_dates = valid_dates_filtered_StringList.getInfo()
  num_valid_dates = len(valid_dates)
  # print("valid_dates", num_valid_dates, valid_dates)

  return valid_dates, TgtCounty_Bnds, TgtCounty_Name


# Function creates retreival path and exports csv's

In [ ]:

def ET_retrieval_path_by_county(county_dict, ET_retrieval_years):
  GEOID = copy.deepcopy(county_dict)

  TgtCounty_GEOID = list(GEOID.keys())[0]
  #start_date_str = str(in_year) + '-01-01' #'2013-01-01'
  # print(TgtCounty_GEOID)


  for current_year in ET_retrieval_years:
    
    start_date_str = str(current_year) + '-01-01' #'2013-01-01'

    for TgtCounty_GEOID in GEOID.keys():
      #valid_dates, TgtCounty_Bnds, TgtCounty_Name = parameter_fn(county_dict, TgtCounty_GEOID, start_date_str)
      valid_dates, TgtCounty_Bnds, TgtCounty_Name = parameter_fn(GEOID, TgtCounty_GEOID, start_date_str)

      # print("TgtCounty_Bnds", TgtCounty_Bnds)
      print("TgtCounty_Name:", TgtCounty_Name)
      # print("valid_dates", valid_dates)

      list_metrics = [] # empty list

      for county_dates in valid_dates:
        #TODO: adjust asset id to have landsat sensor in the parth
        ### check_replace_this_var ###
        asset_folder = 'Ls5'
        # asset_folder = 'C_965_test'
        ETretrieval_Image_path = 'projects/ColoradoView/ET_Seasonal_cPara/' + str(TgtCounty_Name) + '/' + asset_folder + '/' + str(county_dates)
        # ETretrieval_Image_path = os.path.join('projects/ColoradoView/ET_Retrieval', str(TgtCounty_Name), asset_folder, str(county_dates))
        print(ETretrieval_Image_path)
        # ET_retrieval_county_fn(valid_dates, TgtCounty_Bnds, TgtCounty_Name)
        flag_exist = check_asset_existence(str(ETretrieval_Image_path))
        # print(date, flag_exist)

        if not flag_exist:
          print(
            f"Warning: ET retrieval for {TgtCounty_Name} on {county_dates} doesn't exist. "
            "Skip the date."
          )
          continue
    
        scale=30
        metric = metric_dictionary( \
          county_dates, TgtCounty_Bnds, scale, ETretrieval_Image_path)
        
        list_metrics.append(metric)

      # Post processing after valid dates for current year

      # print("Metric list: ", list_metrics)
      # Variable converting empty 'list_metrics' into an ee.List
      ee_list_metrics = ee.List(list_metrics)
      # print("ee_list_metrics: ", ee_list_metrics)

      def create_feature(metricDictionary):
      # Creates a feature from ee_list_metrics
        ret_ft = ee.Feature(None, metricDictionary)
        return ret_ft

      ee_listMetrics_fts = ee_list_metrics.map(create_feature)
      # print(ee_listMetrics_fts)

      metric_to_ftCollection = ee.FeatureCollection(ee_listMetrics_fts)

      desc1 = 'Metrics_table_' + TgtCounty_Name + '_' + str(current_year) # + '-' + end_date_str
      # TODO ...
      # include GEOID, County name
      task = ee.batch.Export.table.toDrive(
        collection=metric_to_ftCollection, 
        description=desc1, 
        fileNamePrefix=desc1,
        folder='ET_cCoef_trials',
        fileFormat='CSV'
      )

      task.start()

In [ ]:
ET_retrieval_years = np.arange(2000, 2001, 1)
print(ET_retrieval_years)

county_dict = {# '08001': 'Adams',
# '08003': 'Alamosa',
# '08005': 'Arapahoe',
# '08007': 'Archuleta',
# '08009': 'Baca',
# '08011': 'Bent',
# '08013': 'Boulder',
# '08015': 'Chaffee',
# '08017': 'Cheyenne',
# '08019': 'Clear_Creek',
# '08021': 'Conejos',
# '08023': 'Costilla',
# '08025': 'Crowley',
# '08027': 'Custer',
# '08029': 'Delta',
# '08031': 'Denver',
# '08033': 'Dolores',
# '08035': 'Douglas',
# '08037': 'Eagle',
# '08041': 'El_Paso',
'08039': 'Elbert',
# '08043': 'Fremont',
# '08045': 'Garfield',
# '08047': 'Gilpin',
# '08049': 'Grand',
# '08051': 'Gunnison',
# '08053': 'Hinsdale',
# '08055': 'Huerfano',
# '08057': 'Jackson',
# '08059': 'Jefferson',
# '08061': 'Kiowa',
# '08063': 'Kit_Carson',
# '08067': 'La_Plata',
# '08065': 'Lake',
# '08069': 'Larimer',
# '08071': 'Las_Animas',
# '08073': 'Lincoln',
# '08075': 'Logan',
# '08077': 'Mesa',
# '08079': 'Mineral',
# '08081': 'Moffat',
# '08083': 'Montezuma',
# '08085': 'Montrose',
# '08087': 'Morgan',
# '08089': 'Otero',
# '08091': 'Ouray',
# '08093': 'Park',
# '08095': 'Phillips',
# '08097': 'Pitkin',
# '08099': 'Prowers',
# '08101': 'Pueblo',
# '08103': 'Rio_Blanco',
# '08105': 'Rio_Grande',
# '08107': 'Routt',
# '08109': 'Saguache',
# '08111': 'San_Juan',
# '08113': 'San_Miguel',
# '08115': 'Sedgwick',
# '08117': 'Summit',
# '08119': 'Teller',
# '08121': 'Washington',
# '08123': 'Weld',
# '08125': 'Yuma'
}

ET_retrieval_path_by_county(county_dict, ET_retrieval_years)

[2000]
TgtCounty_Name: Elbert
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-01-19
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-05-10
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-06-02
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-06-18
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-07-04
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-07-13
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-07-20
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-08-05
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-08-21
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-09-15
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-10-08
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-10-17
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-10-24
projects/ColoradoView/ET_Seasonal_cPara/Elbert/Ls5/2000-11-25


# DEBUG

## Asset exists

In [ ]:
import os
import sys
import datetime
from time import strftime, strptime, gmtime
import math
import pprint
import copy

In [ ]:
!pip install 'pyOpenSSL>=0.11'
!pip install earthengine-api

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 3.2MB 8.0MB/s 


In [ ]:
!earthengine authenticate

import ee
ee.Initialize()

image = ee.Image('srtm90_v4')
print(image.getInfo())

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=76k9K-rMLvx_XxvKc8ZUYvS6JxZ-TpaI8CCiqgtRsfQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7Y8J19sxib5kvFMIkYW__zZlbatRovkP0I6yON9hidYz4gcQJotSI

Successfully saved authorization token.
{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', '

In [ ]:
Test_AssetID_not_exist = 'projects/ColoradoView/ET_Retrieval/Denver/2017-01-30'
Test_AssetID_does_exist = 'projects/ColoradoView/ET_Retrieval/Denver/2017-01-31'

In [ ]:
try:
  ret_info_exist = ee.data.getAsset(Test_AssetID_does_exist)
  print(ret_info_exist)
  ret_info_not_exist = ee.data.getAsset(Test_AssetID_not_exist)
  print(ret_info_not_exist)
except ee.ee_exception.EEException as Err:
  #print(sys.exc_info()[0])
  print(Err)


{'type': 'IMAGE', 'name': 'projects/earthengine-legacy/assets/projects/ColoradoView/ET_Retrieval/Denver/2017-01-31', 'id': 'projects/ColoradoView/ET_Retrieval/Denver/2017-01-31', 'updateTime': '2021-04-27T15:10:02.556312Z', 'geometry': {'type': 'Polygon', 'coordinates': [[[-105.11022535606553, 39.91437015281795], [-105.1102363296469, 39.91432576191721], [-105.110234812973, 39.61435573010939], [-105.11009233490594, 39.61423125473787], [-105.10996359622034, 39.614085696514486], [-104.98248076202336, 39.61408580545438], [-104.85507719348342, 39.61408575500275], [-104.72767366369688, 39.614085768937], [-104.60027035830497, 39.61408578252669], [-104.60010868756974, 39.61419542971396], [-104.59991962729185, 39.614294538386545], [-104.599918115522, 39.91430318443817], [-104.60006111123113, 39.91442751298272], [-104.60018258654777, 39.914564658101035], [-104.60024047855212, 39.91457313026904], [-104.72767366369688, 39.91457318061739], [-104.85507719348342, 39.9145731745171], [-104.982480762023

In [ ]:
def check_asset_existence(asset_id_str: str) -> bool:
  try:
    ret_info = ee.data.getAsset(asset_id_str)
    return True
  except ee.ee_exception.EEException as err:
    return False

In [ ]:
print(check_asset_existence(Test_AssetID_not_exist))
print(check_asset_existence(Test_AssetID_does_exist))

False
True


In [ ]:
val1 = np.array([2018])[0]
print(type(val1), val1)
val1_str = str(val1)
print(type(val1_str), val1_str)

<class 'numpy.int64'> 2018
<class 'str'> 2018
